In [ ]:
# ===== CONSTANTS =====
# Legt den Dateinamen für die Zwischenspeicherung (Cache) fest
cache_file = "cache.csv"

# ===== CORE FUNCTIONS =====

"""
Liеst eine CSV-Datei ein und trennt sie in Header (Spaltennamen) und Zeilen auf.

Parameter:
  file (str): Pfad zur CSV-Datei, die eingelesen werden soll.
Rückgabe:
  header (list): Liste der Spaltennamen aus der ersten Zeile.
  lines (list): Liste aller Zeilen der Datei als rohe Textzeilen.
Funktionsablauf:
  1. Öffnet die Datei im Lesemodus.
  2. Liest alle Zeilen in eine Liste ein.
  3. Trennt die erste Zeile vom Rest und zerlegt sie in Spaltennamen.
"""
def get_content(file):
    with open(file, "r") as data:
        lines = data.readlines()  # Jede Zeile als String inklusive Zeilenumbruch
        header = lines[0].strip().split(",")  # Erste Zeile in Spaltennamen aufteilen
    return header, lines

"""
Erzeugt oder überschreibt die Cache-Datei mit gegebenem Header und Zeilen.

Parameter:
  header (list): Liste von Spaltennamen, wird als erste Zeile gespeichert.
  lines (list of lists): Jede Zeile bereits als Liste von Werten.
  file (str): Dateiname für die Ausgabedatei (Standard: 'cache.csv').
Funktionsablauf:
  1. Öffnet (oder erstellt) die Datei im Schreibmodus.
  2. Schreibt den Header als kommagetrennte Zeile.
  3. Iteriert durch die Zeilen und schreibt jede als kommagetrennte Werte.
"""
def generate_cache_file(header, lines, file=cache_file):
    with open(file, "w") as new_file:
        new_file.write(",".join(header) + "\n")
        for line in lines:
            new_file.write(",".join(line) + "\n")

"""
Gibt eine formatierte Tabelle auf der Konsole aus.

Parameter:
  file (str): Pfad zur CSV-Datei, die angezeigt werden soll.
Funktionsablauf:
  1. Liest Header und Zeilen mit get_content().
  2. Berechnet Anzahl der Datenzeilen und Gesamttabellenbreite.
  3. Zeichnet Trennlinien und druckt Header mit fester Spaltenbreite (20 Zeichen).
  4. Druckt jede Zeile formatiert.
Inspiration & Quelle:
  https://stackoverflow.com/questions/70937491/python-flexible-way-to-format-string-output-into-a-table-without-using-a-non-st
"""
def print_formatted_table(file=cache_file):
    header, lines = get_content(file)
    print("\nFormatted Table, from file:", file)
    print(f"With a total of {len(lines)-1} entrys.")
    print("-" * round(23.2 * len(header)))
    print(("| {:<20} " * len(header) + "|").format(*header))
    print("-" * round(23.2 * len(header)))
    for line in lines[1:]:
        values = line.strip().split(",")
        print(("| {:<20} " * len(header) + "|").format(*values))

"""
Filtert Zeilen nach einem bestimmten Spaltenwert und schreibt das Ergebnis in die Cache-Datei.

Parameter:
  column (str): Spaltenname, nach dem gefiltert werden soll.
  value (list of str): Liste der gewünschten Werte.
  file (str): Pfad zur Eingabedatei.
  return_table (bool): Wenn True, wird die gefilterte Tabelle direkt angezeigt.
Funktionsablauf:
 1. Lese Header und Zeilen.
 2. Prüfe, ob die Spalte existiert.
 3. Iteriere Datenzeilen, überprüfe Werte, sammele Treffer.
 4. Schreibe gefilterte Zeilen in Cache-Datei.
 5. Ausgabe der Filterkriterien und optional Tabelle.
"""
def get_rows_by_value(column, value, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    col_index = header.index(column)
    rows = []
    for line in lines[1:]:
        row = line.strip().split(",")
        if row[col_index] in value:
            rows.append(row)
    generate_cache_file(header, rows)
    print(f"\nOnly returned rows with value in column: {column}")
    print(f"For Values: {value}")
    if return_table:
        print_formatted_table()

"""
Sortiert die Tabelle nach einer bestimmten Spalte mithilfe des Bubble-Sorts.

Parameter:
  column (str): Spaltenname zum Sortieren.
  area (list of int): Zwei Werte [Start, Ende] der Zeilenindizes.
  reverse (bool): Aufsteigend (False) oder absteigend (True).
  file (str): Eingabe-/Ausgabedatei.
  return_table (bool): Wenn True, wird die sortierte Tabelle angezeigt.
Quelle & Algorithmus:
  https://www.geeksforgeeks.org/bubble-sort/
"""
def sort_by(column, area, reverse=False, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    if not area or area[1] < 0:
        area = [area[0], len(lines)]
    rows = [line.strip().split(",") for line in lines[area[0]:area[1]]]
    col_index = header.index(column)
    for i in range(len(rows)):
        for j in range(len(rows)-i-1):
            a = rows[j][col_index]
            b = rows[j+1][col_index]
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j+1] = rows[j+1], rows[j]
    generate_cache_file(header, rows)
    print(f"\nSorted by column: {column}")
    print(f"Range: {area[0]} to {area[1]}")
    if return_table:
        print_formatted_table()

"""
Zählt die Gesamtanzahl der 'Menschen' in der Tabelle.

Parameter:
  file (str): Pfad zur Eingabedatei.
Rückgabe:
  total_humans (int): Summe aller Werte in Spalte 'Number'.
"""
def count_humans(file=cache_file):
    header, lines = get_content(file)
    col_index = header.index("Number")
    total_humans = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        total_humans += int(row[col_index])
    return total_humans

"""
Berechnet das Durchschnittsalter der 'Menschen' in der Tabelle.

Parameter:
  file (str): Pfad zur Eingabedatei.
Rückgabe:
  avg_age (float): Durchschnittsalter basierend auf dem aktuellen Jahr (2025).
"""
def average_age(file=cache_file):
    header, lines = get_content(file)
    col_index = header.index("YearOfBirth")
    current_year = 2025
    total_age = 0
    count = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        age = current_year - int(row[col_index])
        total_age += age
        count += 1
    if count == 0:
        print("No data available to calculate average age.")
        return None
    return total_age / count

# ===== MAIN FUNCTION =====
def main():
    cache_file = "cache.csv"
    original_file = "names.csv"
    source_file = original_file
    while True:
        choice = input("\n(filter/stats/sort/reset/quit): ").lower()
        if choice == 'quit':
            break
        # weitere Logik...
